# Artificial Bee Colony Algorithm 

## Inicialización

In [6]:
# from joblib import dump, load

import copy 
import numpy as np 
from random import randint
from random import random
from random import uniform
import operator

## Cálculo de la Aptitud 

In [7]:
def AptitudProporcional(objective,population,case="minimo"):   
  aptitud=list()
  for individuo in population:    
    if case == "minimo":
      aptitud.append(1/objective([individuo]))
    elif case == "maximo":
      aptitud.append(objective([individuo]))
    else:
      aptitud.append(1/objective([individuo]))
  fS=sum(aptitud)
  prob=list()
  for app in aptitud:
    prob.append(float(app/fS))
  return aptitud,prob

## Mecanismos de selección 

### Selección voraz

In [ ]:
def greedySelection(index,oldSolution, newSolution, oldAptitud, newAptitud, counter):
  if newAptitud > oldAptitud: oldSolution = newSolution
  else: counter[index] = counter[index]+1
  return oldSolution, counter

### Selección por Ruleta

In [8]:
def SeleccionRuletaBee (Probabilidades,foodSources,i=False):
  cumulative_sum = list()  
  #Hacer la suma acumulativa
  for index,prob in enumerate(Probabilidades):
    cumulative_sum.append(sum(Probabilidades[index:]))
  #Seleccionar de acuerdo con su probabilidad  
  p = random()  
  for index,cum_sum in enumerate(cumulative_sum):
    if index < len(cumulative_sum)-1:
      if cumulative_sum[index+1] < p and p <= cumulative_sum[index]: 
        OnlookerBee = index #foodSources[index]
        break
    else:
      if 0 < p and p <= cumulative_sum[index]:
        OnlookerBee = index 
        break
  return OnlookerBee

### Selección por Valor Esperado

In [ ]:
def ValorEsperadoBee(Probabilidades,foodSources,i):
  Expected=list()
  OnlookerBee = list()
  n = len(foodSources)
  for proba in Probabilidades:
    Expected.append(proba*n)
  # print(Expected)
  roundExpected = [round(e) for e in Expected]
  for j in range(roundExpected[i]):
    OnlookerBee.append(i)
  return OnlookerBee

## Exploración y explotación 

In [9]:
#Scout Bees mechanism  
def scout(limits): 
  candidateSolution = list()
  for limit in limits:
    candidateSolution.append(uniform(limit[0],limit[1]))    
  return candidateSolution

#Local search for new food sources
def localSearchFood(index, employedBees):
  iguales = True
  while (iguales):
    k = randint(0,len(employedBees)-1)
    if k != index: iguales = False
  j = randint(0,len(employedBees[k])-1)
  PHI = uniform(-1,1) 
  copyBee = copy.deepcopy(employedBees[index])
  copyBee[j] = employedBees[index][j] + PHI*(employedBees[k][j] - employedBees[index][j])    
  return copyBee

## Integración del algoritmo 

In [5]:
def beeColony(problema, numEBees, numOBees, limits, Niterations,seleccion, steps, case="minimo"):
  print('Inicializando algoritmo')
  Historico = list()
  # HistoricoApt = list()
  Best = None
  HistBest = list()

  stepCounter = [0]*numEBees
  #INITIALIZATION PHASE
  employedBees = list()
  for i in range(numEBees):
    employedBees.append(scout(limits))
  aptitud, probabilidad = AptitudProporcional(problema,employedBees,case)
  Historico.append(employedBees)
  # HistoricoApt.append(aptitud)
  
  print('Ejecutando optimizacion con Artificial Bee Colony Algorithm...')
  for n in range(Niterations):
    #EMPLOYED BEES PHASE    
    newCandidateSolutions = list() 
    for i in range(numEBees):
      newCandidateSol = localSearchFood(i, employedBees)
      newApt, newProb = AptitudProporcional(problema,[newCandidateSol],case)     
      selectedSource, stepCounter = greedySelection(i,employedBees[i],newCandidateSol,aptitud[i],newApt[0],stepCounter)
      newCandidateSolutions.append(selectedSource)                      
    employedBees = copy.deepcopy(newCandidateSolutions)
    aptitud, probabilidad = AptitudProporcional(problema,employedBees,case)

    #ONLOOKER BEES PHASES
    # at this point newCandidateSolutions = employedBees
    for i in range(numOBees):
      OnlookerBee = seleccion(probabilidad,employedBees)
      newCandidateSol = localSearchFood(OnlookerBee,employedBees)
      apt, prob = AptitudProporcional(problema,[newCandidateSolutions[OnlookerBee]],case)     
      newApt, newProb = AptitudProporcional(problema,[newCandidateSol],case)     
      selectedSource, stepCounter = greedySelection(OnlookerBee,newCandidateSolutions[OnlookerBee],newCandidateSol,apt,newApt,stepCounter)
      newCandidateSolutions[OnlookerBee] = selectedSource
    employedBees = copy.deepcopy(newCandidateSolutions)   
    aptitud, probabilidad = AptitudProporcional(problema,employedBees,case) 

    Historico.append(employedBees)
    # HistoricoApt.append(aptitud)
    bestIndex, bestApt = max(enumerate(aptitud), key=operator.itemgetter(1))
    newBest = employedBees[bestIndex]
    if len(HistBest)==0 or  bestApt >= Best:
      Best = bestApt
      HistBest.append(newBest)

    #SCOUT BEES PHASE
    # c=0
    for index,step in enumerate(stepCounter):
      if step > steps:
        # c +=1
        employedBees[index] = scout(limits)
        stepCounter[index] = 0
    aptitud, probabilidad = AptitudProporcional(problema,employedBees,case)
  # print(stepCounter)
  # print(c)
  print('Optimizacion finalizada')
  return [Historico,HistBest]

In [32]:
# ModeloRFR = load('ModeloRFR_2.joblib')

In [22]:
# Geometria = [47.5,0.15,198]
# entrada = list()
# for i in np.arange(0.1,0.9,0.1):
#     entrada.append(Geometria+[i])
#     np.array(entrada)
# ModeloRFR.predict(entrada)

In [23]:
# HistorialPob, HistSoluciones = beeColony(ModeloRFR,10,10,[[40,90],[0.1,0.3],[140,200],[0.4,0.7]],100,SeleccionRuletaBee,60,case='maximo')